## ⚒️ Módulo 7 – Testes Automatizados com Frameworks Python

Embora o Selenium seja uma ferramenta de automação, seu uso mais comum é em **testes de software**. Para criar testes robustos, organizados e escaláveis, integramos o Selenium com frameworks de teste como `unittest` (nativo do Python) ou `pytest` (mais moderno e com uma sintaxe mais limpa).

Usaremos o **pytest** por sua simplicidade e poder.

### **Objetivos**

- Integrar Selenium com `pytest`.
- Escrever testes organizados: cada teste é uma função que começa com `test_`.
- Usar **fixtures** do pytest para `setup` (abrir o navegador) e `teardown` (fechar o navegador) de forma limpa.
- Usar **assertions** (`assert`) para verificar se o comportamento da aplicação está correto.
- Gerar e interpretar relatórios simples de testes no console.

### **Estrutura de um Teste com Pytest**
1.  **Instalação:** `pip install pytest selenium`
2.  **Arquivo de Teste:** Crie um arquivo Python com o nome começando com `test_` (ex: `test_login.py`) ou terminando com `_test.py`.
3.  **Fixture de Setup/Teardown:** Uma função marcada com `@pytest.fixture` que prepara o ambiente (cria o `driver`) e o limpa no final (`yield` e depois `driver.quit()`).
4.  **Funções de Teste:** Funções que começam com `test_` e que recebem a fixture como argumento. Dentro delas, usamos `assert` para validar os resultados.
5.  **Execução:** No terminal, navegue até a pasta do projeto e execute o comando `pytest`.

### **Projeto Prático: Criar testes automatizados para o fluxo de login**
Vamos criar um arquivo de teste para o site ["The-Internet"](https://the-internet.herokuapp.com/login) com dois cenários de teste distintos:
1.  Teste de login com credenciais válidas.
2.  Teste de login com credenciais inválidas.

```python
# Salve este código em um arquivo chamado: test_login_flow.py

import pytest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 3. Fixture: Prepara o ambiente antes de cada teste e limpa depois.
@pytest.fixture
def driver():
    """Esta fixture inicializa o WebDriver antes de cada função de teste
    e o fecha após a execução do teste."""
    # --- SETUP ---
    navegador = webdriver.Chrome()
    navegador.get('[https://the-internet.herokuapp.com/login](https://the-internet.herokuapp.com/login)')
    
    # A palavra 'yield' entrega o controle (e o objeto 'navegador') para a função de teste.
    yield navegador
    
    # --- TEARDOWN ---
    # O código após o 'yield' é executado após o término de cada teste.
    print("Fechando o navegador...")
    navegador.quit()

# 4. Função de Teste para o cenário de sucesso
def test_login_sucesso(driver):
    """Verifica se o login com credenciais válidas funciona e redireciona
    para a página segura."""
    print("Executando teste de login com sucesso...")
    driver.find_element(By.ID, 'username').send_keys('tomsmith')
    driver.find_element(By.ID, 'password').send_keys('SuperSecretPassword!')
    driver.find_element(By.CSS_SELECTOR, 'button.radius').click()

    # 5. Assertion: O momento da verificação (a alma do teste)
    wait = WebDriverWait(driver, 10)
    mensagem_sucesso = wait.until(EC.visibility_of_element_located((By.ID, 'flash')))
    
    # Verificamos se a URL contém a palavra 'secure'
    assert 'secure' in driver.current_url
    # Verificamos se a mensagem de sucesso é a esperada
    assert 'You logged into a secure area!' in mensagem_sucesso.text

# 4. Função de Teste para o cenário de falha
def test_login_falha(driver):
    """Verifica se o login com credenciais inválidas exibe uma mensagem de erro
    apropriada."""
    print("Executando teste de login com falha...")
    driver.find_element(By.ID, 'username').send_keys('usuarioerrado')
    driver.find_element(By.ID, 'password').send_keys('senhaerrada')
    driver.find_element(By.CSS_SELECTOR, 'button.radius').click()

    # 5. Assertion: Verificamos se a mensagem de erro aparece
    wait = WebDriverWait(driver, 10)
    mensagem_erro = wait.until(EC.visibility_of_element_located((By.ID, 'flash')))

    assert 'Your username is invalid!' in mensagem_erro.text

# Para executar:
# 1. Salve este arquivo como 'test_login_flow.py'.
# 2. Abra o terminal na pasta onde o arquivo foi salvo.
# 3. Execute o comando: pytest -v
# (O -v significa 'verbose', para uma saída mais detalhada)